In [0]:
query = "SELECT * FROM ecommerce.silver.events WHERE event_type='purchase'"
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('event_type = purchase)
   +- 'UnresolvedRelation [ecommerce, silver, events], [], false

== Analyzed Logical Plan ==
event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string, ingestion_ts: timestamp, source_file: string, event_date: date, price_tier: string
Project [event_time#13324, event_type#13325, product_id#13326, category_id#13327L, category_code#13328, brand#13329, price#13330, user_id#13331, user_session#13332, ingestion_ts#13333, source_file#13334, event_date#13335, price_tier#13336]
+- Filter (event_type#13325 = purchase)
   +- SubqueryAlias ecommerce.silver.events
      +- Relation ecommerce.silver.events[event_time#13324,event_type#13325,product_id#13326,category_id#13327L,category_code#13328,brand#13329,price#13330,user_id#13331,user_session#13332,ingestion_ts#13333,source_file#13334,event_date#13335,price_tier#

In [0]:
spark.sql("""
CREATE OR REPLACE TABLE ecommerce.silver.events_part
USING DELTA
PARTITIONED BY (event_date)
AS
SELECT * FROM ecommerce.silver.events
""")


DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("""
CREATE OR REPLACE TABLE ecommerce.silver.events_part2
USING DELTA
PARTITIONED BY (event_date, event_type)
AS
SELECT * FROM ecommerce.silver.events
""")


DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("""
OPTIMIZE ecommerce.silver.events_part
ZORDER BY (user_id, product_id)
""")


DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [0]:
import time

def bench(sql, runs=3):
    times = []
    for _ in range(runs):
        start = time.time()
        spark.sql(sql).count()
        times.append(time.time() - start)
    return times

q = "SELECT * FROM ecommerce.silver.events WHERE user_id = 12345"
times_base = bench(q, runs=3)
times_base


[0.5334129333496094, 0.36651039123535156, 0.3534574508666992]

In [0]:
q_part = "SELECT * FROM ecommerce.silver.events_part WHERE user_id = 12345"
times_part = bench(q_part, runs=3)
times_part


[0.3933863639831543, 0.344102144241333, 0.3593432903289795]

In [0]:
spark.sql("SELECT * FROM ecommerce.silver.events_part WHERE event_date = '2019-10-10'").explain(True)


== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('event_date = 2019-10-10)
   +- 'UnresolvedRelation [ecommerce, silver, events_part], [], false

== Analyzed Logical Plan ==
event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string, ingestion_ts: timestamp, source_file: string, event_date: date, price_tier: string
Project [event_time#15019, event_type#15020, product_id#15021, category_id#15022L, category_code#15023, brand#15024, price#15025, user_id#15026, user_session#15027, ingestion_ts#15028, source_file#15029, event_date#15030, price_tier#15031]
+- Filter (event_date#15030 = cast(2019-10-10 as date))
   +- SubqueryAlias ecommerce.silver.events_part
      +- Relation ecommerce.silver.events_part[event_time#15019,event_type#15020,product_id#15021,category_id#15022L,category_code#15023,brand#15024,price#15025,user_id#15026,user_session#15027,ingestion_ts#15028,source_file#1

In [0]:
df = spark.table("ecommerce.silver.events_part").cache()
df.count()


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8996027836894935>, line 1
----> 1 df = spark.table("ecommerce.silver.events_part").cache()
      2 df.count()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:2093, in DataFrame.cache(self)
   2092 def cache(self) -> ParentDataFrame:
-> 2093     return self.persist()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:2100, in DataFrame.persist(self, storageLevel)
   2095 def persist(
   2096     self,
   2097     storageLevel: StorageLevel = (StorageLevel.MEMORY_AND_DISK_DESER),
   2098 ) -> ParentDataFrame:
   2099     relation = self._plan.plan(self._session.client)
-> 2100     self._session.client._analyze(
   2101         method="persist", relation=relation, storage_level=storageLevel
   2102     )
   2103     return self

File /databricks/python